# 0 Modelado

## 0-0 Librerias

In [ ]:
# --- Manejo de datos ---
import os                 # para manejo de carpetas y rutas
import pickle             # para guardar/cargar objetos
import pandas as pd       # manejo de dataframes
import numpy as np        # operaciones numéricas

# --- Visualización ---
import matplotlib.pyplot as plt
import seaborn as sns

# --- Preprocesamiento y división ---
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder

# --- Balanceo de clases ---
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.pipeline import Pipeline

# --- Métricas de evaluación ---
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
    confusion_matrix
)

# --- Modelos clásicos ---
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# --- Modelos avanzados ---
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb


## 0-1 Carga de datos preprocesados

In [31]:
"""
Pickle (Python nativo) → conserva tipos exactos y es muy rápido:
Parquet (Columnar, rápido y seguro) → ideal si quieres usar Pandas:
HDF5 (Pandas) → eficiente y soporta tipos:
"""
with open("df_preprocesado.pkl", "rb") as f:
    df = pickle.load(f)

print(f"Dataset cargado con éxito, {len(df)} registros")
df.info()

Dataset cargado con éxito, 4730 registros
<class 'pandas.core.frame.DataFrame'>
Index: 4730 entries, 0 to 5028
Data columns (total 13 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   address                               4730 non-null   category
 1   t1-labels_encoded                     4730 non-null   int64   
 2   t2-labels_encoded                     4730 non-null   int64   
 3   t1-labels                             4730 non-null   category
 4   t2-labels                             4730 non-null   category
 5   num_transfers_capped_log              4730 non-null   float64 
 6   approx_holders_capped_log             4730 non-null   float64 
 7   top10_concentration_proxy_capped_log  4730 non-null   float64 
 8   avg_transfers_per_day_capped_log      4730 non-null   float64 
 9   std_transfers_per_day_capped_log      4730 non-null   float64 
 10  active_days_capped_log             

## 0-2 Balanceo

In [114]:

# Columnas
identificador = ['address']
x_feature = [
    'num_transfers_capped_log', 'approx_holders_capped_log', 'top10_concentration_proxy_capped_log', 
    'avg_transfers_per_day_capped_log', 'std_transfers_per_day_capped_log', 'active_days_capped_log', 
    'age_days_capped_log', 'inactive_days_proxy_capped_log'
]

y = df['t1-labels_encoded']

# --- Split Train / Temp (80%) y Test (20%) ---
X_train, X_temp, y_train, y_temp = train_test_split(
    df[x_feature], y, test_size=0.2, random_state=42, stratify=y
)

# --- Split Temp en Validation / Test (50%-50% de 20% → 10% cada uno) ---
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

""" 
#se balanceo la clase mayoritaria porque era excesiva, y no refleja realmente la realidad debido a falta de datos
#son terabyte de informacion en bigquery y se obtuvo un pequeño porcentaje de carteras debido a recursos finitos
under = RandomUnderSampler(sampling_strategy={0: 17, 1: 17}, random_state=42)
X_val, y_val = under.fit_resample(X_val, y_val)
X_test, y_test = under.fit_resample(X_test, y_test)
"""




# --- Revisar tamaños ---
print("Train:", X_train.shape, "y:", y_train.value_counts())
print("Validation:", X_val.shape, "y:", y_val.value_counts())
print("Test:", X_test.shape, "y:", y_test.value_counts())


Train: (3784, 8) y: t1-labels_encoded
0    3647
1     137
Name: count, dtype: int64
Validation: (473, 8) y: t1-labels_encoded
0    456
1     17
Name: count, dtype: int64
Test: (473, 8) y: t1-labels_encoded
0    456
1     17
Name: count, dtype: int64


In [123]:
print(X_train.head())


      num_transfers_capped_log  approx_holders_capped_log  \
4197                  0.643638                   0.964703   
2834                  4.185259                   4.071325   
934                   0.339263                   0.694863   
2519                  0.667813                   0.964703   
2589                  0.854014                   1.035533   

      top10_concentration_proxy_capped_log  avg_transfers_per_day_capped_log  \
4197                              4.190647                          1.280406   
2834                              2.840586                          2.734874   
934                               4.712178                          0.640717   
2519                              3.037011                          1.558894   
2589                              4.544081                          0.357699   

      std_transfers_per_day_capped_log  active_days_capped_log  \
4197                          1.574432                1.080213   
2834                

In [118]:


# --- Ver tamaño original ---
print(f"Tamaño original X_train: {len(X_train)}, y_train: {len(y_train)}")

# --- Paso 1: Undersample clase mayoritaria a 1000 ---
target_counts = y_train.value_counts()
under_strategy = {0: 600, 1: target_counts[1]}  # solo reducimos la mayoritaria
under = RandomUnderSampler(sampling_strategy=under_strategy, random_state=42)
X_train_under, y_train_under = under.fit_resample(X_train, y_train)

# --- Paso 2: Oversample clase minoritaria con SMOTE ---
# Hacer que la minoritaria alcance ~50% de la mayoritaria resultante
smote = SMOTE(sampling_strategy=0.7, random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train_under, y_train_under)

# --- Revisar distribución ---
print("Distribución original Train:")
print(y_train.value_counts())
print("\nDistribución después de under + SMOTE:")
print(pd.Series(y_train_bal).value_counts())


Tamaño original X_train: 3784, y_train: 3784
Distribución original Train:
t1-labels_encoded
0    3647
1     137
Name: count, dtype: int64

Distribución después de under + SMOTE:
t1-labels_encoded
0    600
1    420
Name: count, dtype: int64


## 0-3 Modelo

In [121]:

X_train_bal, y_train_bal = smote.fit_resample(X_train_under, y_train_under)
# Crear y entrenar el encoder
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
# --- Modelos con hiperparámetros optimizados ---
modelos = {
    # Modelos lineales simples
    "Logistic Regression": LogisticRegression(
        max_iter=5000,       # más iteraciones para converger si hay pocas features
        solver='lbfgs',      # estable y rápido para datasets pequeños
        C=1.0,               # regularización estándar
        random_state=42
    ),
    "Ridge Classifier": LogisticRegression(
        max_iter=5000,
        solver='saga',       # soporta penalización L2 y datasets pequeños
        penalty='l2',        # Ridge regularization para evitar overfitting
        C=1.0,
        random_state=42
    ),

    # Árboles de decisión
    "Decision Tree": DecisionTreeClassifier(
        max_depth=20,        # profundidad para capturar patrones de la minoritaria
        min_samples_split=5, # evita splits con muy pocos datos
        min_samples_leaf=2,  # evita hojas con muy pocos ejemplos
        class_weight='balanced', # balancea clases automáticamente
        random_state=42
    ),
    "Random Forest": RandomForestClassifier(
        n_estimators=1000,   # muchos árboles para estabilidad
        max_depth=20,        # profundidad suficiente para patrones complejos
        min_samples_split=5,
        min_samples_leaf=2,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1            # usa todos los cores disponibles
    ),

    # Boosting
    "XGBoost": xgb.XGBClassifier(
        n_estimators=5000,   # muchos árboles, aprendizaje lento
        max_depth=12,        # profundidad moderada
        learning_rate=0.02,  # más bajo = entrenamiento más estable
        subsample=0.8,       # muestreo de filas para regularización
        colsample_bytree=0.8,# muestreo de columnas para regularización
        scale_pos_weight=len(y_train_bal[y_train_bal==0])/len(y_train_bal[y_train_bal==1]), # corrige desbalance
        random_state=42,
        use_label_encoder=False,
        eval_metric='aucpr', # métrica útil para desbalance
        tree_method='gpu_hist',  # usa GPU si disponible
        predictor='gpu_predictor'
    ),
    "LightGBM": lgb.LGBMClassifier(
        n_estimators=6000,  # muchos árboles para entrenar lento y estable
        max_depth=15,        # suficiente para capturar minoritaria
        learning_rate=0.1,  # muy bajo para estabilidad
        subsample=0.8,       # regularización por filas
        colsample_bytree=0.8,# regularización por columnas
        class_weight='balanced', # balancea clases automáticamente
        random_state=42
    ),

    # Gradient Boosting clásico de sklearn
    "Gradient Boosting": GradientBoostingClassifier(
        n_estimators=2000,   # muchos árboles para entrenar lento
        learning_rate=0.1,  # bajo para aprendizaje estable
        max_depth=10,         # profundidad moderada para evitar overfitting
        subsample=0.8,       # regularización por filas
        random_state=42
    )
}




C:\Users\win10\AppData\Roaming\Python\Python310\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [ ]:


# Crear carpeta img si no existe
if not os.path.exists("img"):
    os.makedirs("img")

# Diccionario para guardar los mejores modelos
mejores_modelos = {}

# Loop de entrenamiento y evaluación
for nombre, modelo in modelos.items():
    print(f"\nModelo: {nombre}")
    
    # Entrenar modelo sobre datos balanceados
    modelo.fit(X_train_bal, y_train_bal)
    
    # Predicción sobre validación
    y_pred = modelo.predict(X_val)
    
    # Métricas globales
    b_acc = balanced_accuracy_score(y_val, y_pred)
    print(f"→ Balanced Accuracy: {b_acc:.4f}")
    
    # Métricas por clase
    precision = precision_score(y_val, y_pred, average=None)
    recall = recall_score(y_val, y_pred, average=None)
    f1 = f1_score(y_val, y_pred, average=None)
    
    for i, cls in enumerate(le.classes_):
        print(f"Clase {cls}: Precision={precision[i]:.2f}, Recall={recall[i]:.2f}, F1={f1[i]:.2f}")
    
    # Matriz de confusión normalizada
    cm = confusion_matrix(y_val, y_pred, normalize='true')
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues",
                xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title(f"Matriz de Confusión - {nombre}")
    plt.xlabel("Predicción")
    plt.ylabel("Valor Real")
    plt.tight_layout()
    
    # Guardar la matriz de confusión
    plt.savefig(f"img/cm_{nombre.replace(' ', '_')}.png")
    plt.close()
    
    # Guardar modelo entrenado
    mejores_modelos[nombre] = modelo

# Guardar todos los modelos en un pickle
with open("mejores_modelos.pkl", "wb") as f:
    pickle.dump(mejores_modelos, f)

print("Todos los modelos y gráficos han sido guardados correctamente")



Modelo: Logistic Regression
→ Balanced Accuracy: 0.7683
Clase 0: Precision=0.99, Recall=0.77, F1=0.87
Clase 1: Precision=0.11, Recall=0.76, F1=0.19

Modelo: Ridge Classifier
→ Balanced Accuracy: 0.7683
Clase 0: Precision=0.99, Recall=0.77, F1=0.87
Clase 1: Precision=0.11, Recall=0.76, F1=0.19

Modelo: Decision Tree
→ Balanced Accuracy: 0.6801
Clase 0: Precision=0.98, Recall=0.77, F1=0.86
Clase 1: Precision=0.09, Recall=0.59, F1=0.15

Modelo: Random Forest
→ Balanced Accuracy: 0.7566
Clase 0: Precision=0.99, Recall=0.87, F1=0.92
Clase 1: Precision=0.15, Recall=0.65, F1=0.25

Modelo: XGBoost
→ Balanced Accuracy: 0.7577
Clase 0: Precision=0.99, Recall=0.87, F1=0.92
Clase 1: Precision=0.15, Recall=0.65, F1=0.25

Modelo: LightGBM
[LightGBM] [Info] Number of positive: 420, number of negative: 600
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000117 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2039
[